In [1]:
%%cmd 
where python


Microsoft Windows [Version 10.0.22621.3296]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\AYUSH\police project>where python
C:\ProgramData\Anaconda3\python.exe
C:\Python312\python.exe
C:\Users\AYUSH\Medical_project\venv\Scripts\python.exe

(base) C:\Users\AYUSH\police project>

In [2]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [3]:
%%cmd 
where python


Microsoft Windows [Version 10.0.22621.3296]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\AYUSH\police project>where python
C:\ProgramData\Anaconda3\python.exe
C:\Python312\python.exe
C:\Users\AYUSH\Medical_project\venv\Scripts\python.exe

(base) C:\Users\AYUSH\police project>

In [4]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [4]:
%%cmd
pip install cmake

Microsoft Windows [Version 10.0.22621.3296]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\AYUSH\police project>pip install cmake
Defaulting to user installation because normal site-packages is not writeable

(base) C:\Users\AYUSH\police project>

In [5]:
%%cmd
python --version

Microsoft Windows [Version 10.0.22621.3296]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\AYUSH\police project>python --version
Python 3.9.13

(base) C:\Users\AYUSH\police project>

In [6]:
%%cmd
pip install "C:/Users/AYUSH\OneDrive/Documents/dlib package/dlib-19.22.99-cp39-cp39-win_amd64.whl"

Microsoft Windows [Version 10.0.22621.3296]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\AYUSH\police project>pip install "C:/Users/AYUSH\OneDrive/Documents/dlib package/dlib-19.22.99-cp39-cp39-win_amd64.whl"
Defaulting to user installation because normal site-packages is not writeable
Processing c:\users\ayush\onedrive\documents\dlib package\dlib-19.22.99-cp39-cp39-win_amd64.whl
dlib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.

(base) C:\Users\AYUSH\police project>

In [7]:
%%cmd
pip install face-recognition

Microsoft Windows [Version 10.0.22621.3296]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\AYUSH\police project>pip install face-recognition
Defaulting to user installation because normal site-packages is not writeable

(base) C:\Users\AYUSH\police project>

In [8]:
import face_recognition
import cv2
import numpy as np

In [9]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate('serviceAccountKey.json')
firebase_admin.initialize_app(cred)

In [ ]:
import face_recognition
import cv2
import numpy as np
import datetime
import base64
from pytz import timezone

db = firestore.client()

def sharpen_image(image, kernel_size=(5, 5), sigma=0.8, amount=.5, threshold=3):
    # Apply Gaussian blur to the image
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)

    # Calculate the difference between the original and blurred images
    mask = cv2.subtract(image, blurred)

    # Scale the mask and add it back to the original image to sharpen
    sharpened = cv2.addWeighted(image, 1 + amount, mask, -amount, threshold)

    return sharpened

# Load the image
input_image_path = "C:/Users/AYUSH/Downloads/WhatsApp Image 2024-02-05 at 23.50.00_ff846210.jpg"
image = cv2.imread(input_image_path)

# Check if the image was loaded successfully
if image is None:
    print("Error: Unable to load the image.")
else:
    # Sharpen the image (you can adjust the parameters as needed)
    sharpened_image = sharpen_image(image, kernel_size=(5, 5), sigma=0.8, amount=.5, threshold=3)

    # Display the original and sharpened images
    cv2.imshow('Original Image', image)
    cv2.imshow('Sharpened Image', sharpened_image)
    

srk_image = face_recognition.load_image_file("C:/Users/AYUSH/Downloads/WhatsApp Image 2024-02-05 at 23.50.00_ff846210.jpg")
srk_image_encoding = face_recognition.face_encodings(srk_image)[0]

#cv2.imshow('srk_image_encoding', srk_image_encoding)

known_face_encodings = [srk_image_encoding]
known_face_names = ["Criminal"]
c=0
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

capture = cv2.VideoCapture(0)

while True:
    ret, frame = capture.read()

    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]
    
    if process_this_frame:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
                if c==0:
                    doc_ref = db.collection('notifications').document()
                    print("criminal detected")
                    current_time = datetime.datetime.now(timezone("Asia/Kolkata"))
                    
                    filename = current_time.strftime("%Y%m%d-%H%M%S") + '.jpg'
                    cv2.imwrite(filename, frame)
                    _, buffer = cv2.imencode('.jpg',frame)
                    base64_str = base64.b64encode(buffer).decode('utf-8')
                    doc_ref.set({
                        'time':current_time,
                        'Criminal ID': 9211,
                        'cam_proof':"data:image/jpeg;base64,"+base64_str,
                        'type': "criminal"
                    })
                c+=1
            face_names.append(name)

    process_this_frame = not process_this_frame
    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
    # Draw a rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (32, 32, 234), 2)
        cv2.line(frame,(left,top+80),(left+20,top+80),(0,0,255),3)
        cv2.line(frame,(left-20,top+80),(left,top+80),(0,0,255),3)
        cv2.line(frame,(left+80,top),(left+80,top-20),(0,0,255),3)
        cv2.line(frame,(left+80,top),(left+80,top-20),(0,0,255),3)
        cv2.line(frame,(left+80,top),(left+80,top+20),(0,0,255),3)
        cv2.line(frame,(right,bottom-80),(right+20,bottom-80),(0,0,255),3)
        cv2.line(frame,(right,bottom-80),(right-20,bottom-80),(0,0,255),3)
        cv2.line(frame,(right-80,bottom),(right-80,bottom+20),(0,0,255),3)
        cv2.line(frame,(right-80,bottom),(right-80,bottom-20),(0,0,255),3)
    # Input text label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (32, 32, 234), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    # Display the resulting image
    cv2.imshow('Video', frame)
    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

criminal detected


In [10]:
import face_recognition
import cv2
import numpy as np
import datetime
from twilio.rest import Client
import base64
from pytz import timezone

db = firestore.client()

srk_image = face_recognition.load_image_file("C:/Users/AYUSH/Downloads/WhatsApp Image 2024-02-05 at 23.50.00_ff846210.jpg")
srk_image_encoding = face_recognition.face_encodings(srk_image)[0]

account_sid = 'AC808ce16b9bafd62a8981f2e2f0c924fe'
auth_token = '679fb3d924bcd37fc5cb0cddb57311d9'
client = Client(account_sid, auth_token)

known_face_encodings = [srk_image_encoding]
known_face_names = ["Missing Child"]
c=0
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

capture = cv2.VideoCapture(0)

while True:
    ret, frame = capture.read()

    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]

    if process_this_frame:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
                if c==0:
                    print("Missing child")
                    print("Name: Ayush")
                    print("Parents name: Amrish Patel")
                    print("Contact no: 6354270450")
                    message = client.messages.create(
                        body = 'We found your missing child at location at Parul University',
                        from_= '+12187789617',
                        to = '+916354270450'
                    )
                    doc_ref = db.collection('notifications').document()
                    current_time = datetime.datetime.now(timezone("Asia/Kolkata"))
                    
                    filename = current_time.strftime("%Y%m%d-%H%M%S") + '.jpg'
                    cv2.imwrite(filename, frame)
                    _, buffer = cv2.imencode('.jpg',frame)
                    base64_str = base64.b64encode(buffer).decode('utf-8')
                    doc_ref.set({
                        'time':current_time,
                        'children ID': 6354,
                        'cam_proof':"data:image/jpeg;base64,"+base64_str,
                        'type': "child"
                    })
                    
                c+=1
            face_names.append(name)

    process_this_frame = not process_this_frame
    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
    # Draw a rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (32, 32, 234), 2)
        cv2.line(frame,(left,top+80),(left+20,top+80),(0,0,255),3)
        cv2.line(frame,(left-20,top+80),(left,top+80),(0,0,255),3)
        cv2.line(frame,(left+80,top),(left+80,top-20),(0,0,255),3)
        cv2.line(frame,(left+80,top),(left+80,top-20),(0,0,255),3)
        cv2.line(frame,(left+80,top),(left+80,top+20),(0,0,255),3)
        cv2.line(frame,(right,bottom-80),(right+20,bottom-80),(0,0,255),3)
        cv2.line(frame,(right,bottom-80),(right-20,bottom-80),(0,0,255),3)
        cv2.line(frame,(right-80,bottom),(right-80,bottom+20),(0,0,255),3)
        cv2.line(frame,(right-80,bottom),(right-80,bottom-20),(0,0,255),3)
    # Input text label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (32, 32, 234), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    # Display the resulting image
    cv2.imshow('Video', frame)
    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

Missing child
Name: Ayush
Parents name: Amrish Patel
Contact no: 6354270450


TwilioRestException: HTTP 400 error: Unable to create record: The 'From' phone number provided (+12187789617) is not a valid message-capable Twilio phone number for this destination